In [1]:
!nvidia-smi

Wed Mar 27 00:29:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:17:00.0 Off |                  Off |
| 30%   59C    P0              87W / 300W |      4MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install git+https://github.com/openai/whisper.git
!pip install pydub==0.25.1
!pip install langchain==0.1.9

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-v_rkam3l
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-v_rkam3l
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 151.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802826 sha256=4a68e73ea486bfbfaa32cb8e00581c9995fda0c4e89da52c014cce9506423e21
  Stored in directory: /tmp/pip-ephem-wheel-cache-zgo_4psf/wheels/1f/1d/98/9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfully built openai-whisper
Looki

In [3]:
import os
import torch
from pydub import AudioSegment
import whisper
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda:1


In [ ]:
folder_path = os.getcwd()

# Iterate over files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".mp4"):
        video_path = os.path.join(folder_path, file_name)

        # Extract audio from the video
        video = AudioSegment.from_file(video_path, format="mp4")
        
        # Create a corresponding mp3 file name
        audio_output_path = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}.mp3")

        # Export audio to mp3
        video.export(audio_output_path, format="mp3")

        print(f"Audio exported for {file_name} to {audio_output_path}")

In [5]:
model = whisper.load_model("large")

In [24]:
!nvidia-smi

Wed Mar 27 01:39:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:17:00.0 Off |                  Off |
| 30%   35C    P8              14W / 300W |  10129MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [39]:
folder_path = os.getcwd()
segments = []

# Iterate over files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".mp3"):
        audio_path = os.path.join(folder_path, file_name)

        # Transcribe the audio
        transcription_result = model.transcribe(audio_path)

        # Append the result to the segments list along with the file name
        for segment in transcription_result['segments']:
            segment['video_name'] = os.path.splitext(file_name)[0]
            segments.append(segment)

# Now, 'result' contains the concatenated transcripts of all mp3 files in the folder
print(segments)

[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.08, 'text': ' OK, good morning, everybody.', 'tokens': [50365, 2264, 11, 665, 2446, 11, 2201, 13, 50519], 'temperature': 0.0, 'avg_logprob': -0.1757056924361217, 'compression_ratio': 1.4761904761904763, 'no_speech_prob': 0.05410107597708702, 'video_name': 'Kint3'}, {'id': 1, 'seek': 0, 'start': 3.08, 'end': 8.52, 'text': ' So last week, we stopped at the question,', 'tokens': [50519, 407, 1036, 1243, 11, 321, 5936, 412, 264, 1168, 11, 50791], 'temperature': 0.0, 'avg_logprob': -0.1757056924361217, 'compression_ratio': 1.4761904761904763, 'no_speech_prob': 0.05410107597708702, 'video_name': 'Kint3'}, {'id': 2, 'seek': 0, 'start': 8.52, 'end': 11.94, 'text': ' how do we calculate the gradients for more complex loss', 'tokens': [50791, 577, 360, 321, 8873, 264, 2771, 2448, 337, 544, 3997, 4470, 50962], 'temperature': 0.0, 'avg_logprob': -0.1757056924361217, 'compression_ratio': 1.4761904761904763, 'no_speech_prob': 0.05410107597708702, 'video_n

In [42]:
len(segments)

6020

In [43]:
# Extracting only the desired keys from each segment
filtered_segments = [{key: segment[key] for key in ['id', 'seek', 'start', 'end', 'text', 'video_name']} for segment in segments]
len(filtered_segments)

6020

In [44]:
import json
import os
from IPython.display import display, HTML

def save_json_to_file(data, filename):
    file_path = "segments_updated.json"
    # Check if the file already exists
    if os.path.exists(file_path):
        print("file already exists")
        # Load existing JSON data from the file
        with open(file_path, "r") as json_file:
            existing_data = json.load(json_file)
    else:
        # If the file doesn't exist, initialize an empty list
        existing_data = []

    # Append the new array to the existing data
    existing_data.extend(filtered_segments)

    # Convert the combined data to JSON format
    json_data = json.dumps(existing_data, indent=4)

    # Write the JSON data back to the file
    with open(file_path, "w") as json_file:
        json_file.write(json_data)

    print(f"JSON file has been updated at {file_path}")

def generate_download_link(filename):
    download_link = f'<a href="{filename}" download>Download json file containing segments.</a>'
    display(HTML(download_link))

# Example usage:
data = filtered_segments
filename = "segments_updated.json"
save_json_to_file(data, filename)
generate_download_link(filename)

JSON file has been updated at segments_updated.json
